# Enable PolyBase
PolyBase must be enabled on each database that it is going to be used from.


In [ ]:
exec sp_configure @configname = 'polybase enabled', @configvalue = 1;
RECONFIGURE;

Check to see if PolyBase is installed and running.

In [ ]:
SELECT SERVERPROPERTY ('IsPolyBaseInstalled') AS IsPolyBaseInstalled;  

## Master Key  

The first thing you have to do to setup PolyBase is to create a master key. The master key is created on a per database basis. It is recommended that you use a different master key for each database.

In [ ]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'abcdefg123456!@#';
GO

In the event you need to restore the database to a different server, you will need to reset the master key. You can do that by using the following.

In [ ]:
OPEN MASTER KEY DECRYPTION BY PASSWORD = 'abcdefg123456!@#';
GO

ALTER MASTER KEY REGENERATE WITH ENCRYPTION BY PASSWORD = 'abcdefg123456!@#';
GO

## Scoped Credentials

In order for PolyBase to access an external database that requires authentication, a user will need to be set up in the external database. Typically the user only needs read level access. There are some data sources where PolyBase can be used to write as well, but in general PolyBase is only used for reading data.

  

Once the user is created in the external system, you will need to create database scoped credentials using that information.

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL ExtCredName WITH IDENTITY = 'ExternalUser', SECRET = 'ExternalPassword';
GO

-- Azure Data Lake Store
--CREATE DATABASE SCOPED CREDENTIAL ADL_User WITH IDENTITY = '<client_id>@\<OAuth_2.0_Token_EndPoint>', SECRET = '<key>';
--GO

In the event you need to update a scoped credentials due to password reset or some other change in credentials, you can run the following code to update the scoped credentials.

In [ ]:
ALTER DATABASE SCOPED CREDENTIAL QpUser WITH IDENTITY = 'ExternalUser', SECRET = 'newPassword';
GO

## External Data Source

We will use our scoped credentials to create an external data source. The external data source in SQL Server stores the metadata about the external data source such as what type of data source it is and what credentials to use to connect to it if any are required.

--- 

### Location

#### AzureStorage

> LOCATION = 'wasbs://@.blob.core.windows.net'

#### Hadoop - HDFS

> LOCATION = 'hdfs://10.xxx.xx.xxx:xxxx'

#### MongoDB

> LOCATION = N'mongodb://[:]

#### ODBC

> LOCATION = N'odbc://localhost' -- Standard ODBC  
> LOCATION = N'odbc://server.domain.local:port#' -- ODBC connection to SAP HANA

Usually requires extra CONNECTION_OPTIONS.

If connecting to SAP HANA for example you will need the following
> CONNECTION_OPTIONS = 'Driver={HDBODBC}; ServerNode = server.domain.local:port#; '.

For connecting to an ODBC source that has a DSN and no credentials you can use something like
> CONNECTION_OPTIONS = 'DSN=LocalCSV' -- or   
> CONNECTION_OPTIONS = 'Driver=Microsoft Access Text Driver (*.txt, *.csv);Dbq=F:\data\files'.

#### Oracle

> LOCATION = N'oracle://[:]'

#### SQL Server

> LOCATION = N'sqlserver://SqlServer'  
> LOCATION = N'sqlserver://db.be123456789.prod.datalake.servername.com:3657' -- AWS instance

When connecting to a named instance, you will additionally need to add the following connection options.

> CONNECTION_OPTIONS = 'Server=%s\InstanceName;UseDefaultEncryptionOptions=false'

#### Teradata
> LOCATION = teradata://<server address>[:<port>]

--- 

### Pushdown

Pushdown is a setting that tells PolyBase whether to push the filtering criteria to the data source or not.

When Pushdown is on, it will in most cases perform the filtering at the data source. There are certain situations where Pushdown does not work.

When Pushdown is off, PolyBase streams the data before any filtering is performed.

An example would be if you have a table that contains 300,000 rows and your query filters it down to 50,000 rows. With Pushdown on, most of the time PolyBase will filter the data at the source and only stream the 50,000 rows. With Pushdown off, PolyBase will stream all 300,000 rows before applying the filters to filter your results down to 50,000.

--- 

### Credential

This is the scoped credentials you created above for the data source.

### ProvideX (ODBC)

Requires install of MAS 90 4.0 ODBC Driver. A ODBC (64-bit) User or System DSN named SOTAMAS90 must be created using the driver.

In [ ]:
CREATE EXTERNAL DATA SOURCE [DataSourceName]
WITH
( 
	LOCATION = 'odbc://localhost',
    CONNECTION_OPTIONS = 'Driver={MAS 90 4.0 ODBC Driver};DSN=SOTAMAS90',
    CREDENTIAL = [ExtCredName],
    PUSHDOWN = OFF
);

### SQL Anywhere

Requires install of SQL Anywhere 17 ODBC driver.

In [ ]:
CREATE EXTERNAL DATA SOURCE [DataSourceName] 
WITH (
    LOCATION = N'odbc://HostServer',
    CONNECTION_OPTIONS = 'DRIVER={SQL Anywhere 17};
        Host=HostServer; 
        Provider =''SAOLEDB.17'';
        ServerName=DbServer;
        DatabaseName=database;',
    CREDENTIAL = [ExtCredName],
	PUSHDOWN = OFF
);

-- HostServer: Virtual Server
-- DbServer: Named instance of the database server

### SQL Server

#### Default Instance


In [ ]:
CREATE EXTERNAL DATA SOURCE [DataSourceName]
WITH (
    LOCATION = N'sqlserver://SQLSERVERNAME', 
    PUSHDOWN = ON, -- On by default
    CREDENTIAL = [ExtCredName]
);

#### Named Instance


In [ ]:
CREATE EXTERNAL DATA SOURCE [DataSourceName]
WITH (
    LOCATION = N'sqlserver://SQLSERVERNAME', 
    PUSHDOWN = ON, -- On by default
    CREDENTIAL = [ExtCredName],
    CONNECTION_OPTIONS = 'Server=%s\InstanceName;UseDefaultEncryptionOptions=false'
);

### SAP HANA (ODBC)

In [ ]:
CREATE EXTERNAL DATA SOURCE [SapHana]
WITH (
    LOCATION = N'odbc://server.domain.local:port#', 
	CONNECTION_OPTIONS = 'Driver={HDBODBC}; ServerNode = server.domain.local:port#; ',
	PUSHDOWN = ON,
    CREDENTIAL = [ExtCredName]
);

## External Tables


### ProvideX (ODBC)

File based database engine

In [ ]:
CREATE EXTERNAL TABLE [local_schema].[local_table]
( 
	[table_id] int
)
WITH (
	LOCATION = 'FileName', --Table/Filename
	DATA_SOURCE = [DataSourceName]
);

### SQL Anywhere

In [ ]:
CREATE EXTERNAL TABLE [local_schema].[local_table]
(
	[table_id] INT
)
WITH
(
	LOCATION = '[database].[schema].[table]',
    DATA_SOURCE = [DataSourceName]
);


### SQL Server

In [ ]:
CREATE EXTERNAL TABLE [local_schema].[local_table]
(
    [table_id] INT
)
WITH (
    LOCATION = N'[database].[schema].[table]', 
    DATA_SOURCE = [DataSourceName]
);

### SAP HANA


In [ ]:
CREATE EXTERNAL TABLE [sap].[Customer]
(
    [table_id] INT
)
WITH (
    LOCATION = N'"database"."schema.TableName"', 
    DATA_SOURCE = [SapHana]
);

## Statistics

Statistics are used to help SQL server be able to query remote systems better through PolyBase. In most simple terms statistics builds a SQL Server side catalog of information about the remote table. Usually you will create statistics on the index columns of tables you are querying so that SQL Server can store informaiton locally about that index column. This dramatically increases the performance of External Table Queries.

In [ ]:
CREATE STATISTICS statistics_name ON [schema].[table] (field_name) WITH FULLSCAN;
GO